In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np

2024-02-07 20:16:15.896231: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 20:16:15.927501: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 20:16:15.927527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 20:16:15.928346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 20:16:15.933117: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 20:16:15.933617: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2024-02-07 20:16:17.474116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-07 20:16:17.474474: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
df = pd.read_csv("../../2023-05-01.csv", nrows=15000)

In [4]:
(df["from_address"] == df["to_address"]).sum()

10

In [5]:
df = df[df["from_address"] != df["to_address"]]

In [6]:
(df["from_address"] == df["to_address"]).sum()

0

In [7]:
df.columns

Index(['block_timestamp', 'nonce', 'from_address', 'to_address', 'value',
       'gas'],
      dtype='object')

In [8]:
len(set(df['from_address']).union(set(df['to_address'])))

12171

In [9]:
df = df[~df.to_address.isna()]
df = df[~df.from_address.isna()]

In [10]:
ids = {}

for i, id in enumerate(set(df['from_address']).union(set(df['to_address']))):
    ids[id] = i

In [11]:
ids

{'0xbe3a390e62fcbd98d679c1e00d654267c096ec8f': 0,
 '0x9904f6e75462845975b12d8a01fc495f4ee6c60c': 1,
 '0xb6539590e3d87df31528fc8bc428ed281da8486e': 2,
 '0xa5780a14d3c3a9a8ed53a5e66ac4ce45463cb098': 3,
 '0xe3bb8dd952438d28c68c60d518fa78dd93b83dd1': 4,
 '0xa2587eab88c69f9232a3d31632981c8df214377d': 5,
 '0x1b61875b2f3f2fecae85d46301a13be07c6d8d1c': 6,
 '0xdf2c823e219a0c50b8fda1338936334003676811': 7,
 '0x89685bd2467ecefd36a659ab4365f172d1ea2509': 8,
 '0x094e3605fd1440f20de19b3fceb78b7262949bc1': 9,
 '0xf02994ec9abf606ec1bc2f04373cba61d53bd6d7': 10,
 '0x5f719da458f04f04d58c6bdc3c5fd8ef6303cebc': 11,
 '0xf5e6b480fed8d150024e27c6d13e177ccfe6dfd2': 12,
 '0x74495a06dd6ec3aeb4d4680d0cfa5b76ba4b4915': 13,
 '0xe260641cc25953baf594a4602a708455ac2b3eb5': 14,
 '0x00001ed6c88c33dfbf91d10bea764ece0fea2d25': 15,
 '0x8fdf284ef410dd9e28d941abf5750c3926dccd44': 16,
 '0xdf24bad5accfe5bb0cf952eeae26ffba58e048aa': 17,
 '0x9c625db08693668be1f0db0409b91e1447331b9e': 18,
 '0xad7982531c0ca7564a468bbb48fafb56bf60d

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, BatchNormalization, Lambda

# el 1 representa a cualquier indice, un tensor de una dimensión
input_aux = Input(1)

# el 128 es arbitrario, podría ser de cualquier valor
x = Embedding(len(ids), 128)(input_aux)

# TODO: investigar que es esto
x = Dense(64, activation='tanh')(x)

# lleva todos los valores entre -1 y 1
output_aux = BatchNormalization()(x)

model_aux = Model(input_aux, output_aux)
model_aux.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 embedding (Embedding)       (None, 1, 128)            1557248   
                                                                 


 dense (Dense)               (None, 1, 64)             8256      
                                                                 
 batch_normalization (Batch  (None, 1, 64)             256       
 Normalization)                                                  
                                                                 
Total params: 1565760 (5.97 MB)
Trainable params: 1565632 (5.97 MB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [13]:
# son los 3 inputs que representan las 3 cabezas de la red, lo que se conoce como siamesa
input_layer_anchor = Input(1)
input_layer_positive = Input(1)
input_layer_negative = Input(1)

x_a = model_aux(input_layer_anchor)
x_p = model_aux(input_layer_positive)
x_n = model_aux(input_layer_negative)

# un tensor es ... investigar bien. Puede ser cualquier cosa, literalmente.
merged_output = Lambda(lambda tensors: tf.stack(tensors, axis=-1))([x_a, x_p, x_n])

model = Model([input_layer_anchor, input_layer_positive, input_layer_negative], merged_output)

In [14]:
from keras.utils import Sequence
import numpy as np

class GeneratorTriplet(Sequence):
    def __init__(self, df, ids, batch_size):
        self.df = df
        self.act_index = 0
        self.ids = ids
        self.batch_size = batch_size
        self.limit = int(np.ceil(len(self.df) / self.batch_size))

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.act_index < self.limit:
            resultado = self.__getitem__(self.act_index)
            self.act_index += 1
            return resultado
        else:
            raise StopIteration

    def __getitem__(self, index):
        # el Anchor es un A, el positive es un B y el negative es un C. Queremos acercar A y B tanto como sea posible alejando a C. Podría suceder que sea un vecino, pero la probabilidad es baja porque es al azar.
        # Si llegara a elegir algún vecino mal, de casualidad, en una siguiente epoch debería corregirse o inclusive en una misma epoch en una siguiente iteración del generador.
        init = index * self.batch_size
        end = (index + 1) * self.batch_size

        # el batch van a ser #batch_size tuplas que representan el anchor y positive. El negative es un random.
        batch = self.df[init:end]

        # me agarro un sample de #batch_size transacciones.
        negative = self.df.sample(len(batch))

        anchor = np.array(batch['from_address'].apply(lambda x: self.ids.get(x)))
        positive = np.array(batch['to_address'].apply(lambda x: self.ids.get(x)))
        negative = np.array(negative['to_address'].apply(lambda x: self.ids.get(x)))
        
        anchor = tf.convert_to_tensor(anchor)
        positive = tf.convert_to_tensor(positive)
        negative = tf.convert_to_tensor(negative)
        
        # el fake target simunla lo que sería aprendizaje supervisado        
        fake_target = tf.convert_to_tensor(np.array([1]*self.batch_size))

        return ([anchor, positive, negative], [fake_target]*3)

In [15]:
generator = GeneratorTriplet(df, ids, 64)

In [16]:
class TripletCustom(tf.keras.losses.Loss):
    def __init__(self, margin=0.2, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin

    def __call__(self, y, y_pred, sample_weight=None):
        anchor, positive, negative =  tf.split(y_pred, num_or_size_splits=3, axis=-1)
        distance_positive = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        distance_negative = tf.reduce_sum(tf.square(anchor - negative), axis=1)

        loss = tf.maximum(distance_positive - distance_negative + self.margin, 0.0)
        return tf.reduce_mean(loss)

loss = TripletCustom()    

model.compile(
    optimizer=Adam(2e-4),
    loss=loss
)

In [17]:
model.fit(generator, epochs=2)

Epoch 1/2
235/235 [==============================] - 5s 16ms/step - loss: 0.4114
Epoch 2/2
235/235 [==============================] - 4s 16ms/step - loss: 0.2438


# Guardo el embedding y los ids

In [18]:
embedding_matrix = model_aux.get_layer(name="embedding").get_weights()
np.array(embedding_matrix).dump("embedding_matrix.npy")

In [20]:
import pickle
with open('ids.pkl', 'wb') as f:
    pickle.dump(ids, f)

In [19]:
np.load("embedding_matrix.npy", allow_pickle=True)

array([[[-0.00755241,  0.01055122, -0.00257764, ...,  0.00712189,
         -0.04574121, -0.04529754],
        [-0.04348439, -0.03781628,  0.001114  , ...,  0.00380954,
          0.04481931,  0.04540687],
        [-0.0403978 ,  0.01886611,  0.02688554, ..., -0.00895341,
          0.03370035, -0.03449598],
        ...,
        [ 0.04724844,  0.04050041,  0.04798098, ..., -0.03555854,
         -0.01610903,  0.01870744],
        [ 0.02424264,  0.00040799, -0.04468834, ...,  0.01127315,
          0.04484734, -0.01192473],
        [-0.02702595, -0.01289864, -0.05039148, ..., -0.01335092,
          0.00978572,  0.00047866]]], dtype=float32)